In [1]:
from sqlalchemy.orm import declarative_base, Mapped, mapped_column
from sqlalchemy import Integer, String

Base = declarative_base()

class FirstTable(Base):
    __tablename__ = "first_table"
    id: Mapped[int] = mapped_column(Integer, primary_key=True, index=True)
    name: Mapped[str] = mapped_column(String, index=True)


In [2]:
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import sessionmaker
import os
from dotenv import load_dotenv
load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL")

engine = create_async_engine(DATABASE_URL, echo=False, future=True)

AsyncSessionLocal = sessionmaker(
    bind=engine,
    class_=AsyncSession,
    expire_on_commit=False,   # avoids post-commit lazy loads
    autoflush=False,
    autocommit=False,
)

async def init_db() -> None:
    async with engine.begin() as conn:
        # run sync DDL in the async engine
        await conn.run_sync(Base.metadata.create_all)


In [3]:
# Cell 4: create + insert + select (all async, all awaited)
from sqlalchemy import select

async def demo_orm():
    await init_db()

    async with AsyncSessionLocal() as session:
        row = FirstTable(name="Bruno")
        session.add(row)
        await session.commit()

        # Because expire_on_commit=False, row.id is usable without triggering a lazy load:
        print("Inserted id:", row.id)

        # Query
        stmt = select(FirstTable).where(FirstTable.name == "Bruno")
        res = await session.execute(stmt)
        alice = res.scalar_one()
        print("Fetched:", alice.id, alice.name)

await demo_orm()


Inserted id: 5
Fetched: 5 Bruno
